In [11]:
top_directory = '/Users/iaincarmichael/Dropbox/Research/law/law-net/'

from __future__ import division

import os
import sys
import time
from math import *
import copy
import cPickle as pickle

# data
import numpy as np
import pandas as pd

# viz
import matplotlib.pyplot as plt


# graph
import igraph as ig


# NLP
from nltk.corpus import stopwords


# our code
sys.path.append(top_directory + 'code/')
from load_data import load_and_clean_graph, case_info
from pipeline.download_data import download_bulk_resource

sys.path.append(top_directory + 'explore/vertex_metrics_experiment/code/')
from make_snapshots import *
from make_edge_df import *
from attachment_model_inference import *
from compute_ranking_metrics import *
from pipeline_helper_functions import *
from make_case_text_files import *
from bag_of_words import *

# directory set up
data_dir = top_directory + 'data/'
experiment_data_dir = data_dir + 'vertex_metrics_experiment/'

court_name = 'scotus'

# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
G = load_and_clean_graph(data_dir, court_name)

# load similarity matrix

In [9]:
similarity_matrix = load_sparse_csr(filename=experiment_data_dir + 'cosine_sims.npz')

In [14]:
with open(experiment_data_dir + 'CLid_to_index.p', 'rb') as f:
    CLid_to_index = pickle.load(f)

# Look at similarities

In [1]:
def get_similarities(similarity_matrix, CLid_A, CLid_B, CLid_to_index):
    
    if len(CLid_A) != len(CLid_B):
        raise ValueError('lists not the same length')
    else:
        N = len(CLid_A)
    
    # list to return
    similarities = [0] * N

    # grab each entry
    for i in range(N):
        # convet CL id to matrix index
        idA = CLid_to_index[CLid_A[i]]
        idB = CLid_to_index[CLid_B[i]]

        similarities[i] = similarity_matrix[idA, idB]
    
    return similarities

In [4]:
CLid_ing = []
CLid_ed = []
for e in G.es:
    
    CLid_ing.append(G.vs[e.source]['name'])
    CLid_ed.append(G.vs[e.target]['name'])

In [ ]:
start = time.time()
sims = get_similarities(S, CLid_ing, CLid_ed, CLid_to_index)
runtime = time.time() - start

# surgery

In [ ]:
CLid_to_index